In [5]:
import numpy as np 
import pandas as pd 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv

# Spotify API Credentials

In [18]:
CLIENT_ID = "7235323b3a9f485e93919778de3d9ef8"
CLIENT_SECRET = "ec9a4483d04040e2b675d14496fdea65"

In [19]:
#authenticate with Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=CLIENT_ID, 
    client_secret=CLIENT_SECRET
))

In [20]:
#function to get artist information
def get_artist_info(artist_name): 
    result = sp.search(q=artist_name, type="artist", limit = 10)
    if result["artists"]["items"]: #to check the search and if you returns an artist
        return result["artists"]["items"][0]["id"]
    return None

In [21]:
def get_all_albums(artist_id):
    albums = []
    offset = 0  # Offset for pagination
    while True:
        # Fetch albums with pagination
        album_data = sp.artist_albums(artist_id, album_type='album', limit=50, offset=offset)
        # If albums are found, add them to the list
        albums.extend(album_data["items"])
        # If there are more albums, move to the next page
        if album_data["next"]:
            offset += 100  # Increment offset by 50
        else:
            break  # break the loop when no more
    return albums

In [28]:
#function to get album and tracks
def album_track(artist_id):
    albums = get_all_albums(artist_id)
    album_info=[]

    for album in albums:
        album_name = album["name"]
        release_date = album["release_date"]
        album_id = album["id"]

        #track info (extract from api??)
        tracks = sp.album_tracks(album_id)["items"]

        for track in tracks:
            track_name =track['name']
            track_number = track['track_number']
            track_duration =track["duration_ms"]/1000
            spotify_url = track['external_urls']["spotify"]
            
            album_info.append({
                "album_name":album_name,
                "album_id":album_id,
                "album_release_date":release_date, 
                "track_name":track_name,
                "track_number": track_number,
                "track_duration": track_duration,
                "spotify_url":spotify_url
        }) #add all the track data to the album data
        
    return album_info

In [ ]:
# import datetime

# def get_tracks_popularity(artist_id):
#     album_info = album_track(artist_id)
    
#     # Sum up popularity for tracks released in the past 2 years
#     total_popularity = 0
#     num_tracks = 0
    
#     for track in album_info:
#         release_date = track["album_release_date"]
#         release_date = datetime.datetime.strptime(release_date, "%Y-%m-%d")
        
#         # Only consider tracks released in the last 2 years
#         if release_date > datetime.datetime.now() - datetime.timedelta(days=730):  # 730 days = 2 years
#             total_popularity += track["popularity"]
#             num_tracks += 1
    
#     avg_popularity = total_popularity / num_tracks if num_tracks > 0 else 0
#     print(f"Average popularity of tracks released in the last 2 years: {avg_popularity}")
#     return total_popularity  # You can return total or average popularity as a proxy for streams


In [29]:
artist_name = "Olivia Rodrigo" 
artist_id = get_artist_info(artist_name)

if artist_id: 
    album_data= album_track(artist_id)
    if album_data:
        keys = album_data[0].keys()
        filename = f"{artist_name}_albun_info.csv"
        with open(filename, "w", newline="",encoding="utf-8") as file:
            writer = csv.DictWriter(file,fieldnames=keys)
            writer.writeheader()
            writer.writerows(album_data)
        print(f"Data saved to {filename}")
    else:
        print(f"No album or track data found for {artist_name}.")
else:
    print(f"Artist {artist_name} not found.")

Data saved to Olivia Rodrigo_albun_info.csv


# get info on my listening history

In [ ]:
import webbrowser
CLIENT_ID = "7235323b3a9f485e93919778de3d9ef8"
CLIENT_SECRET = "ec9a4483d04040e2b675d14496fdea65"
redirect_uri = "http://localhost:1234/redirect"
scope = "user-top-read"

auth_url = f"https://accounts.spotify.com/authorize?client_id={CLIENT_ID}&response_type=code&redirect_uri={redirect_uri}&scope={scope}"

webbrowser.open(auth_url)

True